In [1]:
from scipy.sparse import *
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import datasets
from importlib import reload as rl
import aux
import json  # salvar benchmarks
import timeit
import scipy


In [2]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

news_bunch = datasets.fetch_20newsgroups(subset='all',
                              remove=('headers', 'footers', 'quotes'))
 
X_news_all = TfidfVectorizer().fit_transform(news_bunch.data)

Y_news_all = news_bunch.target


#### Inicializa corpus e também a versão Tfidf dele

In [3]:
def shrink_cp():
    global cp
    cp = [ 
        'o sapo nao lava op peh nao lava porque nao quer', 
        'ele mora la na lagoa não lava o pé porque não quer', 
        'mas que chulé!', 
        'Ele mora lá na lagoa não lava o pé porque não quer' 
    ]
    _complementary_cp()
def expand_cp(reduce_by = 10):
    global cp
    cp = news_bunch.data[::reduce_by]
    _complementary_cp()

def _complementary_cp():
    global mcp, mcopy, mc0, mc0ar
    mcp = TfidfVectorizer().fit_transform(cp)
    mcp
    mcopy = mcp.copy()
    mc0 = mcp[0]
    mc0ar = mc0.toarray()

In [4]:
def time_function(func, *args, **kargs):
    def wrap():
        if kargs:
            return func(*args, kargs)
        else:
            return func(*args)
        
    return timeit.timeit(wrap)

def loop_time_function(n, func, *args, **kargs):
    """" Executa n vezes a função func, passando args e kargs como argumento.
    Seu retorno é a soma dos tempos de execução de cada chamada à função.
    """
    return sum( 
        timeit.repeat(stmt=f'func(*args, **kargs)', 
                      number=1, repeat=n, setup=f'func = {func}', globals=globals())
#         (time_function(func, *args, **kargs) for _ in range(n) ) 
    )
# loop_time_function(1, print, 3)

In [26]:
# expand_cp(reduce_by = 5)

## Código para subtração manual, elemento a elemento
    + executou de forma consistentemente mais lenta que as versões vetorizadas

In [5]:
def manual_sub(cpy=mcopy, dest=mcp, idx = 0):
    global mc0
    mcp[0] - mc0
    mcp[1] - mc0
    mcp[2] - mc0
    mcp[3] - mc0

NameError: name 'mcopy' is not defined

## Subtração ao se converter tudo para np.array foi a mais veloz
    
    Contudo, inviável para conjuntos maiores de dados (alta complexidade de espaço) conforme demonstrado ao se tentar utilizar essa estratégia com o dataset de notícias
    20newsgroup.
```python
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    cpy.toarray() - cpy[idx].toarray()
```

In [45]:
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    return mcp.toarray() - mcp[idx].toarray()


In [8]:
shrink_cp()
SetPreambleOfSubtractors()

In [17]:
print(mcp.toarray() - mcp[0].toarray())
diff = mcp.toarray() - mcp[0].toarray()

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.28199731  0.35767783  0.28199731 -0.10758753  0.
   0.          0.28199731  0.28199731 -0.78935173  0.56399462 -0.26311724
  -0.26311724  0.0603568   0.28199731  0.          0.0603568  -0.26311724]
 [ 0.57735027  0.          0.          0.         -0.33588866  0.
   0.57735027  0.          0.         -0.78935173  0.         -0.26311724
  -0.26311724 -0.16794433  0.          0.57735027 -0.16794433 -0.26311724]
 [ 0.          0.28199731  0.          0.28199731 -0.10758753  0.35767783
   0.          0.28199731  0.28199731 -0.78935173  0.56399462 -0.26311724
  -0.26311724  0.0603568   0.28199731  0.          0.0603568  -0.26311724]]


In [18]:
from numpy import linalg as LA
print(LA.norm(diff, axis=1))


[0.         1.30125515 1.41421356 1.30125515]


In [9]:
# Define algumas funções candidatas a substituir a subtração com conversão direta
# para array a partir de matriz esparsa.

def SetPreambleOfSubtractors():        
    global mc0ToArray, _sub, _vsub, _stupid_vec, _to_sub, _to_sub
    mc0ToArray = mc0.toarray()
    _sub = lambda a,b: a - b
    _vsub = np.frompyfunc(_sub, 2, 1)
    _stupid_vec = np.frompyfunc(_sub_stupid, 2, 1)
    _to_sub = np.arange( 0, mcp.shape[0], dtype=csr_matrix )
    _to_sub[np.array(range(mcp.shape[0]))] = mc0

# SetPreambleOfSubtractors()
# Ok.
def numpy_gambs_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - np.array(mcp.shape[0]*[mcp[idx]])

# Ok, however in 'ramdom' order (correct submatrix)
def vectorizedpy_sub(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub(np.array([*mcp]), mc0)

# Ok. Same as 'vectorized_sub' 
def vectorized_gambs(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub([*mcp], mc0)

# TESTANTO
def _sub_stupid(a, b):
    return a - b

def stupid_method(cpy=mcopy, dest=mcp, idx = 0):
    mc0ar = np.array(mc0)
    return _stupid_vec([*mcp], mc0ar)

# IDEIA ORIGINALL
def original_idea(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp] - _to_sub

# Parece LENTO
def original_lil(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp.tolil()] -np.array([mc0])
# Ok. Simples e FUNCIONA
def simple_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - [mcp[0]]
def simple_sub_rev(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp] - np.array([mcp[0]])


_power_array_of_mat = np.frompyfunc(lambda x: np.power(x.toarray(), 2), 1, 1)
_norm_array_of_mat = np.frompyfunc(lambda x: sparse.linalg.norm(x), 1, 1) 

_norma_direto_sub = np.frompyfunc(lambda x: np.power(scipy.linalg.norm(x.toarray()), 2), 1, 1)

def sub_sparse_sparse(cpy=mcopy, dest=mcp, idx = 0):
    return _norm_array_of_mat( _power_array_of_mat( [*mcp] - np.array([mcp[1]]) ) )

In [10]:
rl(aux)
print(mcp)
# print(t)
print(type(t))
# print(mcp)
norm_vect = np.frompyfunc(lambda x: np.linalg.norm(x), 1, 1)
print(timeit.timeit(stmt='norm_vect(np.array(aux.csr_nonzero_tuple_array(mcp)))', globals=globals(), number=1 ))

  (0, 17)	0.26311724478937853
  (0, 9)	0.7893517343681357
  (0, 4)	0.3358886580732319
  (0, 11)	0.26311724478937853
  (0, 12)	0.26311724478937853
  (0, 13)	0.16794432903661596
  (0, 16)	0.16794432903661596
  (1, 4)	0.2283011273738551
  (1, 13)	0.2283011273738551
  (1, 16)	0.2283011273738551
  (1, 1)	0.2819973115996728
  (1, 7)	0.2819973115996728
  (1, 2)	0.3576778326573981
  (1, 8)	0.2819973115996728
  (1, 3)	0.2819973115996728
  (1, 10)	0.5639946231993456
  (1, 14)	0.2819973115996728
  (2, 6)	0.5773502691896257
  (2, 15)	0.5773502691896257
  (2, 0)	0.5773502691896257
  (3, 4)	0.2283011273738551
  (3, 13)	0.2283011273738551
  (3, 16)	0.2283011273738551
  (3, 1)	0.2819973115996728
  (3, 7)	0.2819973115996728
  (3, 8)	0.2819973115996728
  (3, 3)	0.2819973115996728
  (3, 10)	0.5639946231993456
  (3, 14)	0.2819973115996728
  (3, 5)	0.3576778326573981


NameError: name 't' is not defined

In [213]:
shrink_cp()
# expand_cp(1)
SetPreambleOfSubtractors()

In [ ]:
diff = simple_sub_rev()
x = lambda i: [*mcp] - np.array([mcp[i]])
mat = x(1)
row0 = mat[0]
# print(scipy.linalg.norm( row0[row0.nonzero()]))
def linalg_norm(m):
#     print('hu! ', scipy.linalg.norm(m[m.nonzero()])) # FALHA SE É APENAS '0' A MATRIZ
    return np.linalg.norm(m.data)

vect_norm = np.frompyfunc(linalg_norm, 1, 1)
vect_norm(mat)

In [212]:
print('type(diff): ', type(diff))
print('diff: ', diff)


type(diff):  <class 'numpy.ndarray'>
diff:  [<1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>]


In [199]:
def _smart_norma_direto_sub(x: 'matriz esparsa'):
    x = x[x.nonzero()]
    print("x: ", x)
    print('t(x): ', type(x))
    return scipy.linalg.norm(x) if x.nonzero() else 0

smart_norma_direto_sub_vect = np.frompyfunc(_smart_norma_direto_sub, 1, 1)

# tt = lambda x: np.power(scipy.linalg.norm(x.toarray()), 2)

# r = _norma_direto_sub(t)
# print(r)
# print(tt(t[0]))
# print('t: ', t)
# print('t[1]: ', t[1])
# print('scipy.linalg.norm(t[1].toarray()): ', 
#        scipy.linalg.norm(t[1].toarray()))
# print('normal maldita', _norma_direto_sub(t[1]))

# print('scipy.linalg.norm(t[1]): ', scipy.linalg.norm(t[1]))

In [200]:
print(t)
# print(t.shape)
# print(t[1])
# print(scipy.sparse.linalg.norm(t[1]))

np.frompyfunc(scipy.sparse.linalg.norm, 1, 1)(t)
# np.linalg.norm (t)


print(timeit.timeit(stmt='np.frompyfunc(scipy.sparse.linalg.norm, 1, 1)(t)', globals=globals(), number=1 ))
# print(timeit.timeit(stmt='smart_norma_direto_sub_vect(t)', globals=globals(), number=1 ))

[<1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>
 <1x18 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>]
0.0020957670058123767


In [201]:
print(mcp.shape)
print(mcp[0].shape)


_sub = lambda a,b: a - b
_vsub = np.frompyfunc(_sub, 2, 1)
# Ok, however in 'ramdom' order (correct submatrix)
def vectorizedpy_sub(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub(np.array([*mcp]), mc0)

# vectorized_sub()
# vectorize_toarray()
# np.array([*mcp])

(4, 18)
(1, 18)


In [58]:
benchmark_small = {}
benchmark_big = {}

In [17]:
# print(mcp[0])

# MLE para os dois abaixo...
ArraySub = lambda a, b:  a.toarray() - b
ArraySubQuick = lambda a, b:  a.toarray() - mc0ar

# Testar versão esparsa dos dois acima
ArraySubCsr = lambda a, b:  csr_matrix(a.toarray() - b)
ArraySubQuickCsr = lambda a, b:  csr_matrix(a.toarray() - mc0ar)

frompyfuncArraySub = np.frompyfunc(ArraySub, 2, 1)
frompyfuncArraySubQuick = np.frompyfunc(ArraySubQuick, 2, 1)
frompyfuncArraySubCsr = np.frompyfunc(ArraySubCsr, 2, 1)
frompyfuncArraySubQuickCsr = np.frompyfunc(ArraySubQuickCsr, 2, 1)

# MLE
def frompysub1_mle(cpy=mcopy, dest=mcp, idx = 0):
  return frompyfuncArraySub([*mcp], mc0)

# Bons resultados, mas MLE
def frompysub2_mle(cpy=mcopy, dest=mcp, idx = 0):
  return frompyfuncArraySubQuick([*mcp], mc0)
# Bons resultados, mas MLE
def frompysub3_big(cpy=mcopy, dest=mcp, idx = 0):
  return frompyfuncArraySubCsr([*mcp], mc0)
def frompysub4_big(cpy=mcopy, dest=mcp, idx = 0):
  return frompyfuncArraySubQuickCsr([*mcp], mc0)


In [51]:
expand_cp(reduce_by = 1)

In [71]:
lista = [
    ('numpy_gambs_sub', numpy_gambs_sub),
    ('vectorizedpy_sub', vectorizedpy_sub),
    ('vectorized_gambs', vectorized_gambs),
    ('stupid_method', stupid_method),
    ('original_idea', original_idea),
    ('original_lil', original_lil),
    ('simple_sub', simple_sub),
    ('simple_sub_rev', simple_sub),
    ('numpy_array_sub', numpy_array_sub),
    ('frompysub1_mle', frompysub1_mle),
    ('frompysub2_mle', frompysub2_mle),
    ('frompysub3_big', frompysub3_big),
    ('frompysub4_big', frompysub4_big),
]

to_run = [
#     ('original_idea', original_idea), # OK
#     ('numpy_gambs_sub', numpy_gambs_sub), # OK
#     ('vectorizedpy_sub', vectorizedpy_sub), # OK
#     ('vectorized_gambs', vectorized_gambs), # OK
#     ('stupid_method', stupid_method), # OK
#     ('original_lil', original_lil), # OK
    ('simple_sub', simple_sub), # OK
    ('simple_sub_rev', simple_sub_rev),

#     ('frompysub3_big', frompysub3_big),
#     ('frompysub4_big', frompysub4_big),
#     ('frompysub1_mle', frompysub1_mle),
#     ('frompysub2_mle', frompysub2_mle),
#     ('numpy_array_sub', numpy_array_sub), # MLE
]
RunBenchmark()

rep:   1, num:   1
simple_sub_rev           :             2.590906
simple_sub               :             2.739317


In [50]:
def timeSubtractions(repeat = 100, number = 100, pairs_name_func=to_run):
  d = {}
  for k, v in to_run:
    d[k] = timeit.repeat(
        stmt=v,
        globals=globals(), repeat=repeat, number=number)
  return repeat, number, d

def RunBenchmark(repeat = 1, number = 1):
  # benchmark[big|small] -> nome -> (num, rep) -> [tempos]
    benchmark = benchmark_small if mcp.shape[0] < 1000 else benchmark_big
    for name, _ in to_run:
        if name not in benchmark:
            benchmark[name] = {}

    rep, num , dic = timeSubtractions(repeat = repeat, number=number)
    # print(dic)
    for name, _ in to_run:
        if (rep, num) not in benchmark[name]:
            benchmark[name][(rep, num)] = []
        benchmark[name][(rep, num)].extend( dic[name] )

    # del benchmark
    print(f"rep: {rep:3d}, num: {num:3d}")
    for k,v in sorted(dic.items(), key=lambda x: min(x[1])):
        print(f"{k:25s}: {min(v):20f}")
RunBenchmark()

rep:   1, num:   1
original_idea            :             0.617867


In [ ]:
nn = 1
for _ in range(1):
    if not benchmark.get(nn):
      benchmark[nn] = {}
      for key, _ in to_run:
        benchmark[nn][key] = []
    else:
      d = benchmark[nn]
      for key, fun in to_run:
        if key not in d:
            benchmark[nn][key] = []

    dict_times = {}
    for key, func in to_run:
      globals()['func'] = func
      spent = timeit.repeat(stmt=f'func()', 
                      number=1, repeat=nn, globals=globals())
      dict_times[key] = spent
      benchmark[nn][key].append(spent)



    # dict_times['numpy_sub'] = loop_time_function(nn, numpy_array_sub)
    rl(aux)
    print(dict_times)
    aux.ShowOrderedDictList(dict_times, option='values')
    print('\n')

In [ ]:
rl(aux)
aux.ShowOrderedDictList(benchmark.get(nn, {}), option='values', smallTime=True)
print('====================')
aux.ShowOrderedDictList(benchmark.get(nn, {}), option='values', smallTime=False)